## Create a system mixing SMIRNOFF and non-SMIRNOFF-formatted force fields

This example shows how to create a receptor-ligand `System` where the ligand (toluene) is parametrized with a SMIRNOFF force field and the protein (T4 Lysozyme) is assigned AMBER parameters through the ParmEd library.

### Parametrize a molecule with smirnoff99Frosst

First, we parametrize the ligand (toluene) with the smirnoff99Frosst force field through the usual route

In [1]:
from simtk.openmm.app import PDBFile

from openforcefield.utils import get_data_filename
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField

In [2]:
# Create an OpenFF Topology of toluene from a pdb file.
toluene_pdb_file_path = get_data_filename('molecules/toluene.pdb')
toluene_pdbfile = PDBFile(toluene_pdb_file_path)
toluene = Molecule.from_smiles('Cc1ccccc1')
off_topology = Topology.from_openmm(openmm_topology=toluene_pdbfile.topology,
                                    unique_molecules=[toluene])

# Load the smirnoff99Frosst system from disk.
force_field = ForceField('smirnoff99Frosst.offxml')

# Parametrize the toluene molecule.
toluene_system = force_field.create_openmm_system(off_topology)

and we convert the OpenMM `System` to a ParmEd `Structure` that we'll be able to mix with the protein

<div class="alert alert-block alert-warning">
  <b>Warning:</b> ParmEd's Structure model is inspired by AMBER. Some information in an OpenMM System are not directly translatable into a Structure. In particular, long-range interaction treatment method (e.g., PME, CutoffPeriodic) and parameters (e.g., cutoff and cutoff switching distance, PME error tolerance) are known to be lost during the conversion.
</div>

In [3]:
import parmed

# Convert OpenMM System into a ParmEd Structure.
toluene_structure = parmed.openmm.load_topology(toluene_pdbfile.topology,
                                                toluene_system,
                                                xyz=toluene_pdbfile.positions)

### Create a ParmEd `Structure` of an AMBER-parametrized receptor

We have to create a ParmEd `Structure` of the receptor (T4 Lysozyme) to combine to the toluene `Structure`. Here we assign `amber99sbildn` parameters using OpenMM.

<div class="alert alert-block alert-info">
    <b>Note:</b> If you already have AMBER (prmtop/inpcrd), GROMACS (top/gro), or any other file specifying the protein parameters supported by ParmEd, you can simply load the files directly into a Structure using ParmEd's functionalities. See https://parmed.github.io/ParmEd/html/readwrite.html .
</div>

In [4]:
t4_pdb_file_path = get_data_filename('proteins/T4-protein.pdb')
t4_pdbfile = PDBFile(t4_pdb_file_path)

# Load the AMBER protein force field through OpenMM.
from simtk.openmm import app
omm_forcefield = app.ForceField('amber99sbildn.xml')

# Parameterize the protein.
t4_system = omm_forcefield.createSystem(t4_pdbfile.topology)

# Convert the protein System into a ParmEd Structure.
t4_structure = parmed.openmm.load_topology(t4_pdbfile.topology,
                                           t4_system,
                                           xyz=t4_pdbfile.positions)

### Combine receptor and ligand structures

We can then merge the receptor and ligand `Structure` objects into a single one.

In [5]:
complex_structure = t4_structure + toluene_structure

### Convert back the structure into an OpenMM System or export to AMBER/GROMACS/CHARMM/OpenMM files

Once we have the `Structure` of the complex, we can chose to create a `System` object that we can simulate with OpenMM

In [6]:
from simtk.openmm.app import NoCutoff, HBonds
from simtk import unit

# Convert the Structure to an OpenMM System in vacuum.
complex_system = complex_structure.createSystem(nonbondedMethod=NoCutoff,
                                                nonbondedCutoff=9.0*unit.angstrom,
                                                constraints=HBonds,
                                                removeCMMotion=False)

or export it in some file format to be run later or by a different simulation engine

In [7]:
# Export the System to an OpenMM XML file.
from simtk.openmm import XmlSerializer
with open('complex.xml', 'w') as f:
    f.write(XmlSerializer.serialize(complex_system))

# Export the Structure to AMBER files.
complex_structure.save('complex.prmtop', overwrite=True)
complex_structure.save('complex.inpcrd', overwrite=True)